In [3]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from safetensors.torch import load_file
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

In [5]:
file_path = '/kaggle/input/law-data-merge/law_data_1.csv'
file_path2 = '/kaggle/input/law-data-merge/law_data_2.csv'
df1 = pd.read_csv(file_path)
df2 = pd.read_csv(file_path2)
df = pd.concat([df1,df2])

In [6]:
df['전문'] = df['전문'].fillna('')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

In [9]:
okt = Okt()

In [10]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

In [11]:
def chunk_text(text, chunk_size=128, overlap=32):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

In [12]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

In [13]:
df['processed_text'] = df['전문'].apply(preprocess_text)
df['chunks'] = df['processed_text'].apply(lambda x: chunk_text(x))

In [14]:
chunk_embeddings = []
chunk_to_doc = []

In [15]:
for idx, chunks in enumerate(df['chunks']):
    for chunk in chunks:
        chunk_embeddings.append(encode_text(chunk))
        chunk_to_doc.append(idx)

In [16]:
chunk_embeddings = torch.stack(chunk_embeddings).to(device)

In [17]:
import numpy as np

def search_query(query, df, chunk_embeddings, chunk_to_doc, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0).to(device)

    query_embedding_cpu = query_embedding.cpu()
    chunk_embeddings_cpu = chunk_embeddings.cpu()

    similarities = cosine_similarity(query_embedding_cpu, chunk_embeddings_cpu).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    results = df.iloc[[chunk_to_doc[i] for i in top_indices]].copy()
    results['similarity'] = similarities[top_indices]

    return results[['주문', 'similarity']]

In [23]:
#query = '피고인이 별다른 이유 없이 함께 술을 마시던 피해자 E을 폭행하고, 경찰에게 피고인의 폭행을 목격하였다고 진술한 F를 보복 목적으로 위험한 물건인 장도리로 머리를 가격하여 상해를 입힌 점에 비추어 그 죄책이 가볍지 아니하다. 그러나 피고인이 당심에 이르러 이 사건 범행을 모두 인정하였고 약 5개월에 결친 수감생활을 통하여 자신의 잘못을 통감하고 반성하고 있는 점, 피해자 F에 대한 보복목적 상해는 우발적으로 이루어졌고 상해의 정도도 비교적 경미한 점, 피해자 F가 피고인과 합의하여 피고인에 대한 처벌을 원하지 않고 있는 점, 피고인에게는 이 사건 범행 이외에는 폭력행위로 처벌받은 전과가 없는 점과 그 밖에 피고인의 나이, 가족관계, 전과관계, 성행, 환경, 범행의 동기와 경위, 범행의 수단과 방법, 범행 후의 정황 등 모든 양형조건을 종합하여 보면, 원심이 피고인에게 선고한 형은 무거워 부당하다고 인정된다. 따라서 피고인의 양형부당 주장은 이유 있고, 검사의 양형부당 주장은 이유 없다.'
#원심판결을 파기한다. 피고인을 징역 10월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 유예한다

#query = '피고인이 여러 차례 무면허운전으로 처벌받았을 뿐만 아니라 2017년 같은 죄 등으로 징역형을 선고받아 누범기간 중에 있었음에도 재차 이 사건 무면허운전 범행을 저지른 점, 이로 인하여 체포되어 조사를 받는 과정에서 동생의 이름을 사용하여 사서명, 사문서, 사기명을 각 위조하고 각 행사한 점, 피고인에게 여러 차례 폭력 전과가 있었음에도 재차 이 사건 상해에 나아간 점, 사기 및 컴퓨터등사용사기의 피해액이 약 7,000만 원 가량에 이르는 점 등은 피고인에게 불리한 정상이다. 반면 피고인이 원심에서 대부분의 범행을 인정하였고 당심에서는 범행 모두를 인정하며 반성하고 있는 점, 원심에서 일부 피해자들(피해 금액 430만 원)과 합의하였고 당심에서도 피해자 AK(피해 금액 3,600만 원)과 추가로 합의한 점 등은 유리한 정상이다. 그 밖에 이 사건 범행의 경위, 범행 후의 정황, 피고인의 연령, 성행, 환경 등 이 사건 변론에 나타난 여러 양형 조건을 종합적으로 고려하면, 원심의 형은 너무 무거워서 부당하다고 인정되므로 피고인의 주장은 이유 있고 검사의 주장은 이유 없다.'

#query  = '피고인은 동종범행(사기죄 또는 근로기준법위반죄)으로 여러 차례 처벌받은 전력이 있으면서 또다시 이 사건 범행을 저질렀다. 이 사건 범행은 피고인이 치밀한 계획에 따라 피해자들을 상대로 투자원금과 고율의 확정수익을 지급하겠다고 기망하여 피해자들로부터 투자금 명목으로 돈을 받아 10억 원 이상을 편취한 것으로 다수의 피해자를 상대로 상당 기간에 걸쳐 반복적으로 범행하였고 피해도 크며 피해자들을 기망하기 위한 수단으로 타인 명의의 투자계약서와 증권계좌 잔고증명서를 위조하여 이를 행사하는 등 범행 수법도 불량하여 그 죄질과 범정이 무겁고 중하다. 근로기준법 위반 범행 또한 피해 근로자의 수와 체불금품의 규모가 작지 않은 점에서 그 죄책이 가볍지 않다. 사기 범행으로 인한 피해자 중 다수에 대해 피해 변제나 피해 회복 조치가 제대로 이루어지지 않아 이에 상응하는 엄중한 처벌이 불가피하다. 다만 피고인이 수사기관에 자수하였고 당심에 이르기까지 범행을 모두 인정하면서 잘못을 반성하고 있으며 근로기준법 위반 범행으로 인한 피해가 대부분 회복되었다. 원심에서 사기범행의 피해를 일부 회복한 데 이어 당심에서도 일부 피해자에게 추가적인 피해 회복을 하였고 가장 큰 피해를 본 특정 경제범죄 가중처벌 등에 관한 법률 위반(사기)죄의 피해자 K에게 피고인의 처 명의 임대보증금반환채권 일부를 양도하는 등 피해 변제를 위해 노력하여 K가 피고인에 대한 처벌을 원하지 않는다는 의사를 표시하고 있다. 이러한 사정들과 함께 피고인의 나이, 성행, 환경, 범행 동기와 경위, 수단과 방법, 범행 후 정황 등 기록과 변론에 나타난 모든 양형 조건들을 종합하여 보면, 원심이 선고한 형은 너무 무거워서 부당하다.'

#query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
query = '고양시 일산동구 B에 있는 C 1층에 있는 피해자 D가 운영하는 E에서, 계산대에 올려 놓은 우유 2팩을 바닥으로 던져 우유가 쏟아지게 하고, 소리를 지르는 등 소란을 피워 위력으로써 피해자의 제과점 운영 업무를 방해하였다'
result = search_query(query, df, chunk_embeddings, chunk_to_doc)

In [24]:
print(result)

                                                                                                                      주문  \
654  피고인을 벌금 500,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 100,000원을 1일로 환산한 기간 피고인을 노역장에 유치한다. 피고인에 대하여 위 벌금에 상당한 금액의 가납을 명한다.   
52                                                                                                  피고인을 징역 1년 2개월에 처한다.   
38                                         피고인을 징역 1년에 처한다. 압수된 식칼 1개(울산지방검찰청 2019년 압 제326호)를 몰수한다. 피치료감호청구인을 치료감호에 처한다.   
134                                        피고인을 징역 1년에 처한다. 압수된 식칼 1개(울산지방검찰청 2019년 압 제326호)를 몰수한다. 피치료감호청구인을 치료감호에 처한다.   
680                                                                                                 피고인을 징역 1년 8개월에 처한다.   

     similarity  
654    0.867112  
52     0.738510  
38     0.734260  
134    0.734260  
680    0.731933  


In [30]:
pd.set_option('display.max_colwidth', None)
row_index = 654
print(df.at[row_index, '주문'])

654    피고인을 벌금 500,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 100,000원을 1일로 환산한 기간 피고인을 노역장에 유치한다. 피고인에 대하여 위 벌금에 상당한 금액의 가납을 명한다.
654                                                                               피고인을 징역 10월에 처한다. 배상신청인들의 배상신청을 모두 각하한다.
Name: 주문, dtype: object


In [32]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, RobertaModel
from safetensors.torch import load_file\


from sklearn.metrics.pairwise import cosine_similarity
import torch

# 파일 경로
file_path = '/kaggle/input/law-data-merge/law_data_1.csv'
file_path2 = '/kaggle/input/law-data-merge/law_data_2.csv'
df1 = pd.read_csv(file_path)
df2 = pd.read_csv(file_path2)
df = pd.concat([df1, df2])

# 결측치 처리
df['전문'] = df['전문'].fillna('')
df['양형의 이유'] = df['양형의 이유'].fillna('')

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 경로와 로드
model_path = '/kaggle/input/kosimcse-1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model_state_dict = load_file(f"{model_path}/model.safetensors")
model = RobertaModel.from_pretrained(model_path, state_dict=model_state_dict).to(device)

# Okt 초기화
okt = Okt()

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

# 텍스트를 청크로 분할하는 함수
def chunk_text(text, chunk_size=128, overlap=32):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

# 텍스트 인코딩 함수
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

# 데이터프레임에 전처리된 텍스트와 청크 추가
df['processed_text'] = df.apply(lambda x: preprocess_text(x['전문'] + ' ' + x['양형의 이유']), axis=1)
df['chunks'] = df['processed_text'].apply(lambda x: chunk_text(x))

# 청크 임베딩 생성
chunk_embeddings = []
chunk_to_doc = []

for idx, chunks in enumerate(df['chunks']):
    for chunk in chunks:
        chunk_embeddings.append(encode_text(chunk))
        chunk_to_doc.append(idx)

chunk_embeddings = torch.stack(chunk_embeddings).to(device)

# 쿼리 검색 함수
def search_query(query, df, chunk_embeddings, chunk_to_doc, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0).to(device)

    query_embedding_cpu = query_embedding.cpu()
    chunk_embeddings_cpu = chunk_embeddings.cpu()

    similarities = cosine_similarity(query_embedding_cpu, chunk_embeddings_cpu).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    results = df.iloc[[chunk_to_doc[i] for i in top_indices]].copy()
    results['similarity'] = similarities[top_indices]

    return results[['주문', '양형의 이유', 'similarity']]


In [49]:
# 테스트 쿼리
#query = '고양시 일산동구 B에 있는 C 1층에 있는 피해자 D가 운영하는 E에서, 계산대에 올려 놓은 우유 2팩을 바닥으로 던져 우유가 쏟아지게 하고, 소리를 지르는 등 소란을 피워 위력으로써 피해자의 제과점 운영 업무를 방해하였다'
#query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
#query = '2023. 3. 15 11:30경 서울시 B, C호 피고인의 주거지에서 피해자와 함께 술을 마시던 중 만취한 상태에서 피해자와 욕설 등 말다툼 중에 우발적으로 그곳에 있던 과도(칼날 길이 약 12cm, 전체 길이 약 22.5cm)를 집어 들고 피해자의 아래 배를 1회 찔러 피해자에게 약 3주간의 치료를 요하는 상해를 가하였다. 피고인은 폭력 범행으로 처벌받은 전력 다수 있다.'
query = '피고인은 2025년 1월 5일 오전 11시 20분경, 부산시 B에 위치한 자신의 집에서 피해자 D(남, 25세)와 함께 술을 마시던 중, 말다툼을 하다가 칼(과도)을 집어 들고 피해자의 가슴을 찔렀습니다. 이로 인해 피해자는 약 1주간의 치료가 필요한 상해를 입었으며, 피고인은 위험한 물건을 사용하여 상해를 가한 범행을 하였습니다. 피고인은 범행을 우발적으로 인정하고 있으며, 피해자는 피고인에 대한 처벌을 원치 않는다는 의사를 밝혔습니다.'
result = search_query(query, df, chunk_embeddings, chunk_to_doc)
print(result)

                                                                                                                                          주문  \
948                                                               피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 3년간 위 형의 집행을 유예한다. 압수된 과도 1개(증 제1호)를 몰수한다.   
511                                                       피고인을 징역 1년에 처한다. 다만 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 압수된 식칼 1자루(증 제1호)를 피고인으로부터 몰수한다.   
956                                        피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 40시간의 알코올 치료강의 수강을 명한다. 압수된 증 제2호를 몰수한다.   
510                                                                                       피고인을 징역 6월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다.   
607  피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유예한다. 피고인에게 보호관찰을 받을 것과 80시간의 사회봉사 및 40시간의 폭력치료강의 수강을 명한다. 압수된 과도 1점(증 제1호)을 피고인으로부터 몰수한다.   

                                                                                                                                       

In [43]:
pd.set_option('display.max_colwidth', None)
row_index = 948
print(df.at[row_index, '양형의 이유'])

피고인이 사용한 위험한 물건의 종류와 피해자의 피해 부위 등에 비추어 보면 이 사건 범행의 위험성이 높았던 것으로 보인다. 피고인은 폭력 범행으로 처벌받은 전력이 다수 있음에도 또 다시 이 사건 범행을 저질렀다. 한편, 피고인은 범행을 인정하고 있다. 피고인은 피해자와 술을 마시던 중 만취한 상태에서 우발적으로 이 사건 범행을 저지른 것이고, 결과적으로 피해자의 상해가 심한 정도에 이르진 않은 것으로 보인다. 피해자는 피고인의 처벌을 원치 않는다는 의사를 밝혔다. 위 정상들을 비롯해 피해자와 피고인의 관계, 피해자 처벌불원 의사의 구체적 내용, 피고인의 연령·성행·환경, 범행의 동기·수단·결과, 범행 후의 정황 등 모든 양형 조건을 종합하여 주문과 같이 형을 정한다.
